In [64]:
import pandas
import numpy

In [65]:
mabc = pandas.read_csv('data/monthly_arrivals_by_country_filtered.csv')

In [67]:
fsi = pandas.read_csv("data/fragilestatesindex_joined.csv")

In [69]:
PSQ_TMS = pandas.read_csv("data/PSQ_TMS.csv", skiprows=range(5), na_values="*")

In [71]:
test = PSQ_TMS[PSQ_TMS["Country Name"] == "Afghanistan"]

In [72]:
test.drop(["Origin / Returned from", "Population type"], axis=1,
    inplace=True)

-c:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
import numpy

In [75]:
sums = test.iloc[:,1:].sum()

In [76]:
values = pandas.Series(["Afghanistan"], ["Country Name"]).append(sums)

In [77]:
test_df = pandas.DataFrame(values)

In [78]:
test_df.transpose()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,1.05111e+06,1.2261e+06,3.37648e+06,912292,1.06678e+06,911679,527710,535632,515659,361985,1.31802e+06,1.54837e+06,1.49935e+06,985191


In [79]:
psq_tms_clean = pandas.read_csv("data/PSQ_TMS_filtered.csv")

In [81]:
gdp_info_raw = pandas.read_csv("data/world_bank_gdp/GDP_2011-2015.csv",skiprows=4)

In [83]:
gdp_clean = pandas.read_csv("data/world_bank_gdp/GDP_filtered.csv")

In [85]:
pov_raw = pandas.read_csv("data/world_bank_poverty/Poverty_raw.csv", skiprows=4)

In [87]:
pov_clean = pandas.read_csv("data/world_bank_poverty/Poverty_filtered.csv")

In [89]:
joined_data = pandas.merge(psq_tms_clean[["2013", "Country Name"]], fsi, on="Country Name")

In [90]:
finance_data = pandas.read_csv("data/financial_data.csv")

In [91]:
commodity_data = pandas.read_csv("data/commodities_filtered.csv")

In [93]:
finance_data_filtered = finance_data[["Country", "WEO Subject Code"] +[str(x) for x in range(2000, 2014)]]

In [94]:
finance_data_filtered.replace("n/a", numpy.nan, inplace=True)

-c:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [96]:
finance_data_filtered = finance_data_filtered[finance_data["WEO Subject Code"] == "NGDPD"]

In [97]:
finance_data_filtered.drop("WEO Subject Code", axis=1, inplace=True)

In [98]:
finance_data_filtered.rename(columns={str(col): "GDP (Billions of USD), " + str(col) for col in range(2000, 2014)}, inplace=True)

In [99]:
finance_data_filtered.rename(columns={"Country": "Country Name"}, inplace=True)

In [100]:
joined_data = pandas.merge(joined_data, finance_data_filtered, on="Country Name")

In [101]:
joined_data.rename(columns={"2013" : "Total Refugee Count, 2013"}, inplace=True)

In [103]:
joined_data = joined_data.applymap(lambda x: str(x).replace(",",""))

In [243]:
np_joined = joined_data.drop(["Country Name"], axis=1).as_matrix().astype("float64")
np_joined = np_joined[~numpy.isnan(np_joined).any(axis=1)]

In [200]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVR

from sklearn import metrics

In [235]:
target = np_joined[:,0]
features = np_joined[:,1:]

ftrain, ftest, ttrain, ttest = train_test_split(features, target, test_size=0.2, random_state=41234)

In [232]:
linear_svr = LinearSVR(loss='epsilon_insensitive', epsilon=0)

In [233]:
linear_svr.fit(ftrain, ttrain)
preds_svr = linear_svr.predict(ftest)
print numpy.sqrt(metrics.mean_squared_error(ttest, preds_svr))
print metrics.r2_score(ttest, preds_svr)

261469.617566
-0.048745791616
